In [ ]:
# Perform the DOE and generate the equations to be used


In [13]:
import pandas as pd
import numpy as np
import math

In [18]:
# Calculates the response values for
# h, w, t, L, E
# Units: mm, mm, mm, mm, Pa, N
def generateResponseValues(h, w, t, L, E, F):
    # Fixed Values needed for calculation
    mt = 90.0 # Mass of Tube
    rr = 600 #mm, Distance to center line
    omega = 2 # rev/s, Rotational Speed
    g = 9.81 # m/s^2
    rho = 8.8E3 # kg/m^3
    
    # Calculations
    #Area Moment of Inertia
    wf = w+ 2*t# Flange Width
    I = wf * (math.pow(t, 3)/6.0 + t*math.pow(h+t, 2)/2.0) + t * math.pow(h, 3)/6
    # Tip Load
    #Fstatic = mt * g
    #Fdynamic = mt * (rr/1000) * math.pow(2 * math.pi * omega, 2)
    #Fcalc = Fstatic + Fdynamic # Fmax
    # Deflection
    U = 1E6 * F * math.pow(L, 3)/(3 * E * I)
    # Stress
    M = F * L # Moment
    S = M * (h/2 + t)/I # Stress
    # Beam Mass
    mb = 2E-9 * rho * L * t * (w + 2*t + h)
    # Frequency
    mbar = mt + 0.23 * mb
    a = 0.003*E*I/(mbar*math.pow(L, 3))
    freq = math.sqrt(a)/(2 * math.pi)    
    return U, S, freq


In [2]:
from pyDOE import *


In [49]:
#ccDesign_6Var = ccdesign(6, alpha='o', face='ccf')
ccDesign_6Var = lhs(6, samples=200, criterion='m')
print 'DOE Runs: %d'%(len(ccDesign_6Var))
#print ccDesign_6Var

DOE Runs: 200


In [50]:
# 6 Variables: h, w, t, L, E, F
lb = np.array([80, 40, 2, 400, 180E9, 9000])
ub = np.array([120, 60, 7, 600, 220E9, 11000])
avg = 0.5 * (lb + ub)

nDOE = len(ccDesign_6Var)

# Values in physical space
ccDesign_6Var_PhysicalSpace = np.ones([nDOE, 1]) * avg + np.multiply(np.ones([nDOE, 1]) * (ub - lb) * 0.5, ccDesign_6Var)
h_PS = ccDesign_6Var_PhysicalSpace[:,0]
w_PS = ccDesign_6Var_PhysicalSpace[:,1]
t_PS = ccDesign_6Var_PhysicalSpace[:,2]
L_PS = ccDesign_6Var_PhysicalSpace[:,3]
E_PS = ccDesign_6Var_PhysicalSpace[:,4]
F_PS = ccDesign_6Var_PhysicalSpace[:,5]

deflection_PS = np.zeros([nDOE, 1])
stress_PS = np.zeros([nDOE, 1])
freq_PS = np.zeros([nDOE, 1])

#print ccDesign_6Var_PhysicalSpace
for iDOE in np.arange(nDOE):
    deflection_PS[iDOE,:], stress_PS[iDOE, :], freq_PS[iDOE, :] = generateResponseValues(h_PS[iDOE], w_PS[iDOE], t_PS[iDOE], L_PS[iDOE], E_PS[iDOE], F_PS[iDOE])
#iDOE = 1
#print ccDesign_6Var_PhysicalSpace[iDOE,:]
#a, b, c = generateResponseValues(ccDesign_6Var_PhysicalSpace[iDOE,0], ccDesign_6Var_PhysicalSpace[iDOE,1], ccDesign_6Var_PhysicalSpace[iDOE,2], ccDesign_6Var_PhysicalSpace[iDOE,3], ccDesign_6Var_PhysicalSpace[iDOE,4], ccDesign_6Var_PhysicalSpace[iDOE,5])
#print a, b, c


In [51]:
# Fit the Response Surface on this
# http://scikit-learn.org/0.17/auto_examples/model_selection/plot_underfitting_overfitting.html
import numpy as np
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn import cross_validation

In [52]:
polynomial_features = PolynomialFeatures(degree=2, include_bias=True)
linear_regression = LinearRegression()
pipeline = Pipeline([("polynomial_features", polynomial_features),("linear_regression", linear_regression)])
pipeline.fit(ccDesign_6Var_PhysicalSpace, freq_PS)
print polynomial_features.get_feature_names(['h', 'w', 't', 'L', 'E', 'F'])

# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, freq_PS, scoring="neg_mean_squared_error", cv=10)
print scores

pipeline.fit(ccDesign_6Var_PhysicalSpace, deflection_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, deflection_PS, scoring="neg_mean_squared_error", cv=10)
print scores

pipeline.fit(ccDesign_6Var_PhysicalSpace, stress_PS)
# Evaluate the models using crossvalidation
scores = cross_validation.cross_val_score(pipeline, ccDesign_6Var_PhysicalSpace, stress_PS, scoring="neg_mean_squared_error", cv=10)
print scores


['1', 'h', 'w', 't', 'L', 'E', 'F', 'h^2', 'h w', 'h t', 'h L', 'h E', 'h F', 'w^2', 'w t', 'w L', 'w E', 'w F', 't^2', 't L', 't E', 't F', 'L^2', 'L E', 'L F', 'E^2', 'E F', 'F^2']
[-0.3429767  -0.32941599 -0.46687081 -0.68039152 -0.46554866 -0.50946248
 -0.32012006 -0.64140965 -0.4012813  -0.84835925]
[-0.00161568 -0.0010771  -0.0017065  -0.00202625 -0.00130462 -0.00049856
 -0.00077868 -0.00096058 -0.00074602 -0.00135376]
[-3.18232465 -3.72576111 -5.57648678 -7.65550949 -3.22141352 -1.68440346
 -2.5399686  -3.24667493 -4.20076022 -5.69932215]
